# Description

TODO

# Modules

In [1]:
import pandas as pd

# from tqdm import tqdm
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

pandas2ri.activate()

# from clustermatch.utils import simplify_string
from clustermatch import conf

In [2]:
clusterProfiler = importr("clusterProfiler")

# Settings

In [3]:
# OUTPUT_DIR = conf.GTEX["DATA_DIR"] / "data_by_tissue"
# OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# display(OUTPUT_DIR)

# Data loading

In [4]:
input_filename = conf.GTEX["DATA_DIR"] / "gtex_gene_id_symbol_mappings.pkl"
display(input_filename)

PosixPath('/opt/data/data/gtex_v8/gtex_gene_id_symbol_mappings.pkl')

In [5]:
data = pd.read_pickle(input_filename)

In [6]:
data.shape

(56200, 2)

In [7]:
data.head()

,gene_ens_id,gene_symbol
0,ENSG00000145309.5,CABS1
1,ENSG00000175820.3,CCDC168
2,ENSG00000083454.21,P2RX5
3,ENSG00000071794.15,HLTF
4,ENSG00000211918.1,IGHD2-15


# Add Ensembl IDs without version

In [8]:
data = data.rename(columns={"gene_ens_id": "gene_ens_id_v"})

In [9]:
data = data.assign(gene_ens_id=data["gene_ens_id_v"].apply(lambda x: x.split(".")[0]))

In [10]:
data = data[["gene_ens_id_v", "gene_ens_id", "gene_symbol"]]

In [11]:
data.head()

,gene_ens_id_v,gene_ens_id,gene_symbol
0,ENSG00000145309.5,ENSG00000145309,CABS1
1,ENSG00000175820.3,ENSG00000175820,CCDC168
2,ENSG00000083454.21,ENSG00000083454,P2RX5
3,ENSG00000071794.15,ENSG00000071794,HLTF
4,ENSG00000211918.1,ENSG00000211918,IGHD2-15


In [12]:
assert data.shape[0] == data.drop_duplicates().shape[0]

# Add Entrez Gene IDs

In [13]:
assert data["gene_ens_id_v"].is_unique

In [14]:
data["gene_ens_id"].is_unique

False

In [15]:
data["gene_symbol"].is_unique

False

Gene Ensembl IDs (without version) and gene symbols by their own are not unique.

In [16]:
gene_ens_ids = data["gene_ens_id"].unique().tolist()
display(len(gene_ens_ids))
display(gene_ens_ids[:5])

56156

['ENSG00000145309',
 'ENSG00000175820',
 'ENSG00000083454',
 'ENSG00000071794',
 'ENSG00000211918']

In [17]:
entrez_gene_ids = clusterProfiler.bitr(
    gene_ens_ids,
    fromType="ENSEMBL",
    toType="ENTREZID",
    OrgDb="org.Hs.eg.db",
    drop=True,
)

R[write to console]: Loading required package: org.Hs.eg.db

R[write to console]: Loading required package: AnnotationDbi

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, m

In [18]:
entrez_gene_ids.shape

(34496, 2)

In [19]:
entrez_gene_ids.head()

,ENSEMBL,ENTREZID
1,ENSG00000145309,85438
2,ENSG00000175820,643677
3,ENSG00000083454,5026
4,ENSG00000071794,6596
5,ENSG00000211918,28503


In [20]:
assert entrez_gene_ids.shape[0] == entrez_gene_ids.drop_duplicates().shape[0]

In [21]:
entrez_gene_ids["ENSEMBL"].drop_duplicates().shape

(34259,)

In [22]:
entrez_gene_ids["ENTREZID"].drop_duplicates().shape

(34407,)

In [23]:
entrez_gene_ids = entrez_gene_ids.rename(
    columns={
        "ENSEMBL": "ensembl_id",
        "ENTREZID": "entrez_id",
    }
)

# Merge

In [24]:
gene_mappings = pd.merge(
    data[["gene_ens_id_v", "gene_ens_id"]],
    entrez_gene_ids,
    left_on="gene_ens_id",
    right_on="ensembl_id",
    how="inner",
)[["gene_ens_id_v", "ensembl_id", "entrez_id"]]

In [25]:
gene_mappings.shape

(34527, 3)

In [26]:
gene_mappings.head()

,gene_ens_id_v,ensembl_id,entrez_id
0,ENSG00000145309.5,ENSG00000145309,85438
1,ENSG00000175820.3,ENSG00000175820,643677
2,ENSG00000083454.21,ENSG00000083454,5026
3,ENSG00000071794.15,ENSG00000071794,6596
4,ENSG00000211918.1,ENSG00000211918,28503


# Save

In [27]:
output_filename = conf.GTEX["DATA_DIR"] / "gtex_entrez_gene_ids_mappings.pkl"
display(output_filename)

PosixPath('/opt/data/data/gtex_v8/gtex_entrez_gene_ids_mappings.pkl')

In [28]:
gene_mappings.to_pickle(output_filename)